## Parse w32_mal_report and hooklogs
* w32_mal_report is from Virustotal (total 357 malware/binaries)
* hooklohgs are from dynamic analysis (total 420 files/processes)

In [38]:
import os
import json

w32_mal_report = 'data/w32_mal_report/'
hooklogs = 'data/hooklogs/'

# MIKE: if text_mining is True, the result strings will be stripped.
text_mining = True
csv_file = 'output/w32_mal_report_with_hooklog_tm.csv' if text_mining else 'output/w32_mal_report_with_hooklog.csv'

In [39]:
import string
def _isHex(s):
    if len(s) == 0:
        return False
    for c in s:
        if c not in string.hexdigits:
            return False
    return True

### w32_mal_report
First run, get malware list

In [40]:
malware_file_list = next(os.walk(w32_mal_report))[2]
mal_in_rep_list = list()
for r in malware_file_list:
    mal_in_rep_list.append(r[:-len('.txt')])

print 'mal_in_rep_list', len(mal_in_rep_list) # number of malware in the w32_mal_report

#=======

# But not all the malware in the report is Win32, so ...
for f in malware_file_list:
    h = open(w32_mal_report + f, 'r')
    json_report = h.read()
    if h: h.close()
    malware = f[:-len('.txt')]
    
    d = json.loads(json_report)
#     if(d['type'] != 'Win32 EXE'):
#         print malware, 'is not win32. Remove from mal_in_rep_list.'
#         mal_in_rep_list.remove(malware)
    if(d['type'] == 'Win32 EXE' or d['type'] == 'Win32 DLL' or d['type'] == 'PDF'):#=====10/5
        pass
    else:
        print malware, 'is not win32. Remove from mal_in_rep_list.'
        mal_in_rep_list.remove(malware)
        
print 'mal_in_rep_list', len(mal_in_rep_list) # number of malware in the w32_mal_report

mal_in_rep_list 845
fff7e9818eb72a69a74574b82c9ac750 is not win32. Remove from mal_in_rep_list.
ffc2a9fb284620fade1f62c5108a33a0 is not win32. Remove from mal_in_rep_list.
fefe8519f9313566498edcecea364930 is not win32. Remove from mal_in_rep_list.
fff35334a0d544b0e3993abc2424e080 is not win32. Remove from mal_in_rep_list.
fffc57a2432febbc36918c9035691570 is not win32. Remove from mal_in_rep_list.
ff0afb92de67d0774b585bb5ac539dc0 is not win32. Remove from mal_in_rep_list.
ffffc91eb0599adcab179e662e151d60 is not win32. Remove from mal_in_rep_list.
ff2fcc87eb85d149d05e6b0ac0297f00 is not win32. Remove from mal_in_rep_list.
fff9ab0617b7ed51f661873a1b0ddbb0 is not win32. Remove from mal_in_rep_list.
fff14238f5f8c79c7de26e25be5ed380 is not win32. Remove from mal_in_rep_list.
fff6ff8eee7a73bbc1a53b4755679db0 is not win32. Remove from mal_in_rep_list.
fffdf5e46f8b83025c4f2efc1ed32b20 is not win32. Remove from mal_in_rep_list.
ffb1f59b3af724522951c94891b50430 is not win32. Remove from mal_in_re

### hooklogs
First run, get process list and the malware list. Note that the some hooklog has shorten name.

In [41]:
process_file_list = next(os.walk(hooklogs))[2]
proc_in_log_list = list()
mal_in_log_set = set()
for p in process_file_list:
    m_p = p[:-len('.trace.hooklog')]
    proc_in_log_list.append(m_p)
    mal_in_log_set.add(m_p.split('_')[0])

print 'mal_in_log_set', len(mal_in_log_set) # number of malware in the hooklog
print 'proc_in_log_list', len(proc_in_log_list) # number of processes in the hooklog

mal_in_log_set 684
proc_in_log_list 1282


### Intersection
I have no idea which list is correct, so make an intersection to make sure the malware/process list is correct.

In [42]:
#MIKE: if hooklog's name is not shorten, the line below is good enough. However, ...
#Find correct malware, than correct process
#inter_list = set(mal_in_rep_list).intersection(mal_in_log_set)
#print len(inter_list)

# Make a dict, d[sort name] = long_name; Make a true process list
proc_list = proc_in_log_list
m_set = mal_in_log_set
mal_dict = dict()

#os.path.getsize(hooklogs + process + '.trace.hooklog') < 2000:

for m in list(m_set): # could be short name
    _done = False
    for n in mal_in_rep_list: # long name    #==============not in rep 不用for
        if m == n:
            mal_dict[m] = n
            _done = True
            break
        elif m in n:
            mal_dict[m] = n
            _done = True
            break
    if not _done: # cannot find corresponding item
        print 'no such', m, 'in report.'
        m_set.remove(m)

for m_p in proc_in_log_list:
    m = m_p.split('_')[0]
    if m not in mal_dict.keys():
        proc_list.remove(m_p)

print 'mal_dict', len(mal_dict) # MIKE: the malware we want to deal with
print 'proc_list', len(proc_list) # MIKE: the corresponding processes

no such eab7dffde4bfe178afe646077541d650 in report.
no such ff503cd80e9b8a58645cd15b0bfb3df4 in report.
no such ffb1f59b3af724522951c94891b50430 in report.
no such ff0afb92de67d0774b585bb5ac539dc0 in report.
no such ff300db3b586161fad806a08e1070430 in report.
no such ff6b463d771c6778318a0169671ee310 in report.
no such ffc3fa4a7dd1d3872e990afbe0036d00 in report.
no such fff6a14a112f7d4f0e332c77a528d830 in report.
no such ff79e2e6737db896925946fa9551d9c0 in report.
mal_dict 675
proc_list 1273


In [43]:
#=======
# 1) check again, using long_name
# 2) do text_mining ds at the same time

import re
tag_dict = dict()
delimiter = '\,|!|\(|\)|\[|\]|@|:|/|\.| '

av_set = set()
for malware in mal_dict.values():
    h = open(w32_mal_report + malware + '.txt', 'r')
    json_report = h.read()
    if h: h.close()
    
    d = json.loads(json_report)
    for c, s in enumerate(d['scans'].keys()):
        scan_result = d['scans'].get(s)
        if scan_result.get("detected") == True:
            engine = s.encode('ascii', 'ignore')
            result = scan_result.get("result").encode('ascii', 'ignore')
            result = result.replace(',', '') # special replacement for csv
            av_set.add(engine)
            
            if text_mining:
                tokens = re.split(delimiter, result.lower())
                for k in tokens:
                    if len(k) == 0:
                        continue
                    if k in tag_dict:
                        tag_dict[k] += 1
                    else:
                        tag_dict[k] = 1

av_list = list(av_set)
print 'av_list', len(av_list), av_list

av_list 59 ['Bkav', 'TotalDefense', 'MicroWorld-eScan', 'nProtect', 'CMC', 'CAT-QuickHeal', 'ALYac', 'Malwarebytes', 'Zillya', 'AegisLab', 'TheHacker', 'K7GW', 'K7AntiVirus', 'NANO-Antivirus', 'F-Prot', 'Symantec', 'Norman', 'ESET-NOD32', 'TrendMicro-HouseCall', 'Avast', 'ClamAV', 'GData', 'Kaspersky', 'BitDefender', 'Agnitum', 'ViRobot', 'ByteHero', 'Tencent', 'Ad-Aware', 'Emsisoft', 'Comodo', 'F-Secure', 'DrWeb', 'VIPRE', 'AntiVir', 'TrendMicro', 'McAfee-GW-Edition', 'Sophos', 'Cyren', 'Jiangmin', 'Avira', 'Antiy-AVL', 'Kingsoft', 'Arcabit', 'SUPERAntiSpyware', 'Microsoft', 'Commtouch', 'AhnLab-V3', 'McAfee', 'AVware', 'VBA32', 'Baidu-International', 'Zoner', 'Rising', 'Ikarus', 'Fortinet', 'AVG', 'Panda', 'Qihoo-360']


In [44]:
#Special handel for tag_dict
general_string = ['win32','trojan','adware','generic','application','variant','downloader','not-a-virus','downware',
                 'unwanted-program','heur','troj','bundler','antifw','riskware','optional','malware','behaveslike',
                 'kcloud','agent','trojandownloader','appl','trojware','installer','trojan-downloader','virus',
                 'backdoor','injector','malware-cryptor','dropper','cryptor','bundleapp','suspicious','antifwk',
                 'adinstaller','crypt','bundleinstaller','xpack', 'hacktool','patcher','troj_gen','grayware',
                 'software','install','click','heuristic','packed','unknown','applicunwnt','dropped','trojan-clicker',
                 'net-worm','monitoringtool','worm','tool','toolbar','eldorado','autorun','hw32', 'trojan-dropper']
special_string = ['kdz', 'ipz', 'lmn']
tag_key_list =  tag_dict.keys()

In [45]:
# Remove tags that have only 1 count, or ...

for k in tag_key_list:
    if  k in special_string:
        continue
    elif _isHex(k):
        del tag_dict[k]
    elif tag_dict[k] <= 1 or len(k) <= 3:
        del tag_dict[k]
    elif k in general_string:
        del tag_dict[k]
     
print 'tag_dict', len(tag_dict), tag_dict.keys()

tag_dict 1902 ['multplug', 'a-5f58ee87', 'pay-per', 'bdmj', 'browsefox', 'sixer', 'gcgn', 'delf', 'ekjk', 'adload', 'gen-zbot', 'spambot', 'cqkwjb', 'gen-ganel', 'a-f5597a5b', 'conjar', 'qoys', 'troj_cutwail', '$emu', 's-947a495c', 'a-a2151e6a', 'sytro-ac', 'sytro-ab', '30in13', 's-55467851', 'r047c0fjt14', 'adware-hotbar', 'loring', 'awz+7hhkvka', 'cryp_yodac', 'badfile', 'a-fb629e6f', 'gen-flooder', '1p142tkyjlg', 'sisproc', '0bga14', 'firseria-c', 'a-6d9fc912', 'worm_vobfus', 'lydrak', 'vitro', 'dcygsx', 'hfsiemusi', 'zegost-a', 'alusinus', 'heuristic-131', 'pakes', 'winsxsbot', 'androm', 'altbrowse', 'aona', 'aone', 'agentcv', 'dnascan', 'elenoocka', 'cmio', 'autoit', 'pincav', 'clicker', 'ricd', 'generic36', 'generic35', 'troj_injecto', 'cycgal', 'madangel', 'hcjw', 'threat-hllsi-based', 'aamt', 'delphi', 'r047c0cln14', 'a-7393b86c', 'installco', 'emgfbcgpf0j', 'troj_downloader_cd102ff4', 'durta', 'genus', 'covkiz', 'pinball', 'madang', 'nofer', 'oyxu', 'damaged', 'cvthxc', 'bkdr_

### Make csv and pickle files

In [46]:
o = open(csv_file, 'w')
# header
o.write("%s,%s,%s,%s,%s" % ('Hash', 'Type', 'First_Seen', 'Scan', 'Tag'))
map(lambda s: o.write(',%s' % s), av_list)
o.write('\n')

malwareFami_dict={}    #==========

#each malware
for malware in mal_dict.values():
    h = open(w32_mal_report + malware + '.txt', 'r')
    json_report = h.read()
    if h: h.close()
    
    d = json.loads(json_report)
    # meta for each malware
    o.write("%s,%s,%s,%d/%d" % (malware, d['type'], d['first_seen'], d['positives'], len(d['scans'])))
    
    # meta Tag, no ','
    tags = ''
    this_tag_set = set()
    
    # arrange result
    candidate_dict = dict()
    for c, s in enumerate(d['scans'].keys()):
        scan_result = d['scans'].get(s)
        if scan_result.get("detected") == True:
            engine = s.encode('ascii', 'ignore')
            result = scan_result.get("result").encode('ascii', 'ignore')
            candidate_dict[engine] = result
            
    # output av result for this malware
    out = ''
    for av in av_list:
        if av in candidate_dict:
            result = candidate_dict[av].replace(',', '') # special replacement for csv
            if text_mining:
                result_tag = ''
                tokens = re.split(delimiter, result.lower())
                for k in tokens:
                    if k in tag_dict:
                        if k not in this_tag_set:
                            this_tag_set.add(k)
                            tags += (k+';')
                        result_tag += (k+'.')
                if result_tag:
                    result_tag = result_tag[:-1] # remove last .
                out += (',' + result_tag)
            else:
                out += (',' + result)
        else:
            out += ','
    if tags:
        tags = tags[:-1] # remove last ;
    
#     print malware, tags.split(';')[0]  #==========
#     malwareFami_dict[malware] = tags.split(';')[0]  #==========
    if len(tags.split(';'))>=2: 
        malwareFami_dict[malware] = tags.split(';')[1]  #==========
    else: 
        malwareFami_dict[malware] = ''  #==========
    print malware, malwareFami_dict[malware]  #==========
    o.write(',' + tags + out + '\n')

if o: o.close()
    
import pickle
with open('pickle/av_list-mal_dict-family_dist.pickle', 'wb') as o:
    pickle.dump(malwareFami_dict, o)
if o: o.close()

ffffa131fa245abae651fa71ca9a4f60 kazy
fff98513f7dc548029ff5e741b7569c0 megasearch
ff10655612f7162511911d73d2c8d882 spyware
fffc5416a5cf167ee6f27842013b9b20 firseria
fffc990c40d720aac37eb8e808f693d0 eggogka
ffe2f3a42a55a25f537e466735c48eb0 urelas
fff7e8d58ce9442f2a31cccdb4b54140 sprotector
ffffb6ff6afe83a660bdab06ef16fea0 clkmbpd
ff08082645665f33630663500a8262d0 webber
ffbb1651c7ed93e50d52ffce98b7eec4 installrex
ffeea9b553453bb1c1e29612d6d30890 kazy
fff8cb46c4a78741cf6727bb4ed09430 pondfull
ffdb3c00f7cc7c25f8240eed3a1d4a50 viking
fff351ad66140a5e49eb323c4bf53700 msil
fffb4b1134fac1215a52741c00ca8a40 bettersurf
ffe5c541f680e4833789b825d25b4410 downloadware
fff41d5d937463575f001ce28bee59a0 simbot
ff03829a123e0b133b270b4067e62280 zbotk
ffdc76ac04d7b8f9bd848d2695468e50 ribkrh8o1tciy
ffb2b412efb8d79e8e932ed7997a0111 otwycal
ffff3a8233f091edde59b1321933d970 infoleak
fffc3af2e56bb74a8bfdee5da8580000 eggogka
ffea4eb7a22be131b1ac2348bccee131 androm
ff46a6b614f15c680b913af2c8728cc0 ramnit
fffb9e4

In [47]:
import pickle

data = [av_list, mal_dict, proc_list]

with open('pickle/av_list-mal_dict-proc_list.pickle', 'wb') as o:
    pickle.dump(data, o)